# This notebook aims to convert crime_rate external dataset's geospatial data to SA2_code

In [2]:
import requests
import zipfile
import os
import geopandas as gpd

# Define download URLs
sa2_url = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip"
lga_url = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/LGA_2021_AUST_GDA2020_SHP.zip"

# Directory where the data will be saved
output_dir = '../../data/landing/External_data/lga_to_sa2'

# Create folder if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define paths to save downloaded files
sa2_zip_path = os.path.join(output_dir, "SA2_2021_AUST_SHP_GDA2020.zip")
lga_zip_path = os.path.join(output_dir, "LGA_2021_AUST_GDA2020_SHP.zip")

# Define paths to extract the zip files
sa2_extract_path = os.path.join(output_dir, "SA2_2021_AUST_SHP")
lga_extract_path = os.path.join(output_dir, "LGA_2021_AUST_SHP")

# Function: download file
def download_file(url, file_path):
    print(f"Downloading {url} ...")
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=128):
                file.write(chunk)
        print(f"Saved as: {file_path}")
    else:
        print(f"Failed: {response.status_code}")

# Function: unzip file
def unzip_file(zip_file_path, extract_to_path):
    print(f"Uncompressing {zip_file_path} ...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
    print(f"Successfully decompressed: {extract_to_path}")

# Download SA2 Shapefile
download_file(sa2_url, sa2_zip_path)

# Download LGA Shapefile
download_file(lga_url, lga_zip_path)

# Unzip SA2 Shapefile
unzip_file(sa2_zip_path, sa2_extract_path)

# Unzip LGA Shapefile
unzip_file(lga_zip_path, lga_extract_path)


Saved as: ../data/landing/External_data/lga_to_sa2/SA2_2021_AUST_SHP_GDA2020.zip
Saved as: ../data/landing/External_data/lga_to_sa2/LGA_2021_AUST_GDA2020_SHP.zip
Uncompressing ../data/landing/External_data/lga_to_sa2/SA2_2021_AUST_SHP_GDA2020.zip ...
Successfully decompressed: ../data/landing/External_data/lga_to_sa2/SA2_2021_AUST_SHP
Uncompressing ../data/landing/External_data/lga_to_sa2/LGA_2021_AUST_GDA2020_SHP.zip ...
Successfully decompressed: ../data/landing/External_data/lga_to_sa2/LGA_2021_AUST_SHP


In [1]:
import geopandas as gpd


lga_boundaries = gpd.read_file("../data/landing/External_data/lga_to_sa2/LGA_2021_AUST_SHP/LGA_2021_AUST_GDA2020.shp")
sa2_boundaries = gpd.read_file("../data/landing/External_data/lga_to_sa2/SA2_2021_AUST_SHP/SA2_2021_AUST_GDA2020.shp")


lga_boundaries = lga_boundaries.to_crs(sa2_boundaries.crs)


In [6]:
import geopandas as gpd

# Perform spatial join using the 'intersects' predicate to find overlapping areas between LGA and SA2
lga_to_sa2 = gpd.sjoin(lga_boundaries, sa2_boundaries, how="inner", predicate="intersects")

# Check the result
print(lga_to_sa2.head())


  LGA_CODE21 LGA_NAME21 STE_CODE21_left  STE_NAME21_left AUS_CODE21_left  \
0      10050     Albury               1  New South Wales             AUS   
0      10050     Albury               1  New South Wales             AUS   
0      10050     Albury               1  New South Wales             AUS   
0      10050     Albury               1  New South Wales             AUS   
0      10050     Albury               1  New South Wales             AUS   

  AUS_NAME21_left  AREASQKM21_left  \
0       Australia         305.6386   
0       Australia         305.6386   
0       Australia         305.6386   
0       Australia         305.6386   
0       Australia         305.6386   

                                     LOCI_URI21_left  SHAPE_Leng  SHAPE_Area  \
0  http://linked.data.gov.au/dataset/asgsed3/LGA2...    1.321768     0.03056   
0  http://linked.data.gov.au/dataset/asgsed3/LGA2...    1.321768     0.03056   
0  http://linked.data.gov.au/dataset/asgsed3/LGA2...    1.321768     0.030

In [7]:
# Drop the 'geometry' column before saving as CSV since CSV doesn't support geometry data
lga_to_sa2 = lga_to_sa2.drop(columns="geometry")

# Save the result to a CSV file
lga_to_sa2.to_csv("../data/landing/External_data/LGA_to_SA2_mapped.csv", index=False)


In [16]:
import pandas as pd

# Load crime data
crime_data = pd.read_csv("../data/landing/External_data/LGA_criminal_incidents/LGA_criminal_incidents_table_1.csv")

# Load LGA to SA2 mapping data
lga_to_sa2 = pd.read_csv("../data/landing/External_data/LGA_to_SA2_mapped.csv")

# Standardize LGA names in crime data
crime_data['Local Government Area'] = crime_data['Local Government Area'].str.strip().str.lower()

# Standardize LGA names in LGA to SA2 mapping data
lga_to_sa2['LGA_NAME21'] = lga_to_sa2['LGA_NAME21'].str.strip().str.lower()

# Filter the LGA to SA2 mapping data for Victoria (assuming the column 'STE_NAME21' represents the state name)
lga_to_sa2_victoria = lga_to_sa2[lga_to_sa2['STE_NAME21_left'] == 'Victoria']

# Merge the crime data with Victoria's LGA to SA2 mapping table
merged_data = pd.merge(crime_data, lga_to_sa2_victoria, how="left", left_on="Local Government Area", right_on="LGA_NAME21")

# Select the columns to keep: SA2-related columns and crime data columns such as 'Rate per 100,000 population'
selected_columns = ['Local Government Area', 'Incidents Recorded', 'Rate per 100,000 population', 
                    'SA2_CODE21', 'SA2_NAME21']

# Create a new dataframe containing the selected key columns
final_data = merged_data[selected_columns]

# Check the result
print(final_data.head())

# Save the merged data to a CSV file
final_data.to_csv("../data/landing/External_data/victoria_crime_data_with_sa2_and_rates.csv", index=False)
print("The merged Victoria data has been saved to 'victoria_crime_data_with_sa2_and_rates.csv'.")


  Local Government Area  Incidents Recorded  Rate per 100,000 population  \
0               banyule                6168                  4633.858403   
1               banyule                6168                  4633.858403   
2               banyule                6168                  4633.858403   
3               banyule                6168                  4633.858403   
4               banyule                6168                  4633.858403   

    SA2_CODE21                 SA2_NAME21  
0  206071145.0  Clifton Hill - Alphington  
1  207011148.0               Balwyn North  
2  207011154.0                   Kew East  
3  206021110.0     Alphington - Fairfield  
4  209011201.0   Ivanhoe East - Eaglemont  
The merged Victoria data has been saved to 'victoria_crime_data_with_sa2_and_rates.csv'.
